In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import MeanSquaredError
import cv2
import numpy as np
import os
from glob import glob
from sklearn.model_selection import train_test_split

# Load images function
def load_images(image_dir, image_size=(256, 256)):
    images = []
    for img_path in glob(os.path.join(image_dir, "*.png")):  # Assuming images are in PNG format
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, image_size)
            img = img / 255.0  # Normalize to [0, 1]
            images.append(img)
    print(f"Loaded {len(images)} images from {image_dir}")
    return np.array(images)

# Load clean images (HR) and noisy images (LR)
X_train = load_images(r'C:\Users\ASUS\Downloads\DIV2K_train_HR\DIV2K_train_HR')
y_train = load_images(r'C:\Users\ASUS\Downloads\DIV2K_train_LR\DIV2K_train_LR')

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Build the Denoising Model
def build_denoising_model():
    model = Sequential()
    model.add(Input(shape=(256, 256, 3)))  # Input layer
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(3, (3, 3), activation='sigmoid', padding='same'))

    # Compile the model with the MeanSquaredError function directly
    model.compile(optimizer='adam', loss=MeanSquaredError())
    return model

# Instantiate the model
model = build_denoising_model()
model.summary()  # View the model architecture

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_val, y_val))

# Save the model for later use
model.save('denoising_model.h5')

Loaded 800 images from C:\Users\ASUS\Downloads\DIV2K_train_HR\DIV2K_train_HR
Loaded 800 images from C:\Users\ASUS\Downloads\DIV2K_train_LR\DIV2K_train_LR


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 256, 256, 3)    │           867 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,763 (6.89 KB)

 Trainable params: 1,763 (6.89 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 4s 89ms/step - loss: 0.0767 - val_loss: 0.0534
Epoch 2/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 85ms/step - loss: 0.0469 - val_loss: 0.0226
Epoch 3/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 85ms/step - loss: 0.0194 - val_loss: 0.0109
Epoch 4/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step - loss: 0.0107 - val_loss: 0.0089
Epoch 5/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step - loss: 0.0085 - val_loss: 0.0074
Epoch 6/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step - loss: 0.0076 - val_loss: 0.0068
Epoch 7/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 83ms/step - loss: 0.0071 - val_loss: 0.0062
Epoch 8/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 83ms/step - loss: 0.0063 - val_loss: 0.0059
Epoch 9/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step - loss: 0.0060 - val_loss: 0.0056
Epoch 10/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step - loss: 0.0059 - val_loss: 0.0054
Epoch 11/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step - loss: 0.0056 - val_loss: 0.0052
Epoch 12/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 83ms/step - loss: 0.0